In [18]:
from Bio import Entrez
from Bio.Seq import Seq
from Bio.Data import CodonTable

Entrez.email = "qamilmirza@berkeley.edu"  # Required for NCBI queries

# Search for the species in the NCBI taxonomy database
species_name = "Hansenula saturnus"  # Change to any species of interest
handle = Entrez.esearch(db="taxonomy", term=species_name)
record = Entrez.read(handle)
handle.close()

if record["IdList"]:
    taxon_id = record["IdList"][0]
    print(f"Taxonomy ID: {taxon_id}")

    # Fetch taxonomy details to obtain the genetic code ID
    handle = Entrez.efetch(db="taxonomy", id=taxon_id, retmode="xml")
    records = Entrez.read(handle)
    handle.close()

    if records:
        genetic_code = records[0].get("GeneticCode", {})
        gc_id = genetic_code.get("GCId", "N/A")
        gc_name = genetic_code.get("GCName", "N/A")
        print(f"Genetic Code ID: {gc_id}")
        print(f"Genetic Code Name: {gc_name}")

        # Print the associated codon table
        if gc_id.isdigit():
            gc_id = int(gc_id)
            if gc_id in CodonTable.unambiguous_dna_by_id:
                codon_table = CodonTable.unambiguous_dna_by_id[gc_id]
                print("\nCodon Table Retrieved:")
                print(codon_table)
            else:
                print(f"Codon table for Genetic Code ID {gc_id} is not available in Biopython.")
        else:
            print("Invalid Genetic Code ID.")
    else:
        print("No records found.")
else:
    print("Species not found.")

Taxonomy ID: 907340
Genetic Code ID: 1
Genetic Code Name: Standard

Codon Table Retrieved:
Table 1 Standard, SGC0

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA Stop| A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I   | ACT T   | AAT N   | AGT S   | T
A | ATC I   | ACC T   | AAC N   | AGC S   | C
A | ATA I   | ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   |

In [1]:
# Advanced Project Code: Species-Dependent Translator
from Bio.Data import CodonTable

def translate_sequence(seq, species="universal"):
    """
    Translate a DNA sequence using the codon table appropriate for the given species.
    
    Parameters:
        seq (str): The DNA sequence (should be in 5'-to-3' orientation).
        species (str): Either "universal" or a string indicating another species/organellar code.
                       For this example, if species is "yeast_mito", we use table 3.
    
    Returns:
        str: The translated protein sequence.
    """
    # Choose the appropriate codon table.
    # (This mapping can be expanded or automated by querying NCBI taxonomy data.)
    if species.lower() in ["saccharomyces cerevisiae mitochondria", "yeast_mito", "yeast mitochondria"]:
        table_id = 3  # Yeast mitochondrial code (as defined by NCBI)
    else:
        table_id = 1  # Universal code
    
    codon_table = CodonTable.unambiguous_dna_by_id[table_id]
    
    protein = ""
    # Process the sequence in windows of three.
    for i in range(0, len(seq), 3):
        codon = seq[i:i+3]
        # Ignore incomplete codons at the end.
        if len(codon) != 3:
            continue
        # Check if the codon is a stop codon.
        if codon in codon_table.stop_codons:
            protein += "*"
        else:
            # Look up the amino acid. If the codon is not found, use 'X' for unknown.
            aa = codon_table.forward_table.get(codon, "X")
            protein += aa
    return protein

# Example usage:
dna_seq = "ATGAAGACCTGG"  # Example sequence; adjust as needed.
protein_universal = translate_sequence(dna_seq, species="universal")
protein_yeast = translate_sequence(dna_seq, species="yeast_mito")

print("Universal translation:", protein_universal)
print("Yeast mitochondrial translation:", protein_yeast)


Universal translation: MKTW
Yeast mitochondrial translation: MKTW


In [2]:
dna_seq = "ATGATAGTATTAGGTAAGTGA"
protein_universal = translate_sequence(dna_seq, species="universal")
protein_yeast = translate_sequence(dna_seq, species="yeast_mito")

print("Universal translation:", protein_universal)
print("Yeast mitochondrial translation:", protein_yeast)


Universal translation: MIVLGK*
Yeast mitochondrial translation: MMVLGKW


In [1]:
import requests
from Bio.Data import CodonTable
from Bio import Entrez
from Bio import SeqIO
from io import StringIO
from urllib.parse import quote 

def get_species_codon_table(species_name):
    """
    Input: species_name (str) - the name of the species to search for
    Output: table_id (int) - the genetic code table ID for the species
    """
    encoded_species = quote(species_name)
    search_url = f"https://www.ebi.ac.uk/ena/taxonomy/rest/scientific-name/{encoded_species}"
    
    try:
        response = requests.get(search_url)
        response.raise_for_status()
        data = response.json()
        print(data)
        
        if isinstance(data, list) and len(data) > 0:
            taxon_info = data[0]
            return int(taxon_info.get("geneticCode", 1))  # Ensure integer
        else:
            return None
    except requests.RequestException as e:
        return None

In [12]:
from Bio import Entrez, SeqIO

def get_mrna_sequence(species_name, gene_name, email="qamilmirza@berkeley.edu"):
    """
    Fetch mRNA sequences from NCBI for a given species and gene name.

    Parameters:
        species_name (str): Name of the species (e.g., "Homo sapiens")
        gene_name (str): Name of the gene (e.g., "INS")
        email (str): Your email for NCBI Entrez (required)

    Returns:
        dict: {accession_number: mRNA_sequence (with T->U conversion)}
    """
    Entrez.email = email  # Set email for NCBI Entrez API
    
    try:
        # New query: using property filters for mRNA and RefSeq records
        search_query = (f"{species_name}[Organism] AND {gene_name}[Gene] "
                        "AND biomol_mrna[PROP] AND srcdb_refseq[PROP]")
        print(f"🔍 Search Query: {search_query}")  # Debug: show search query

        # Increase retmax if needed; here we limit to 5 records
        handle = Entrez.esearch(db="nucleotide", term=search_query, retmax=5)
        record = Entrez.read(handle)
        handle.close()
        
        print(f"✅ Found IDs: {record['IdList']}")  # Debug: show list of IDs

        if not record["IdList"]:
            print("❌ No mRNA sequences found. Try a different query or check gene name.")
            return {}

        mrna_sequences = {}

        for gene_id in record["IdList"]:
            print(f"📥 Fetching sequence for ID: {gene_id}...")
            fetch_handle = Entrez.efetch(db="nucleotide", id=gene_id, rettype="fasta", retmode="text")
            fasta_data = fetch_handle.read()
            fetch_handle.close()

            fasta_lines = fasta_data.strip().split("\n")
            # The header is the first line; remove the initial '>' and extract accession (first token)
            header = fasta_lines[0].split()[0][1:]
            sequence = "".join(fasta_lines[1:])  # Join all sequence lines together
            
            # Convert DNA to RNA (T -> U)
            rna_sequence = sequence.replace("T", "U").replace("t", "u")
            mrna_sequences[header] = rna_sequence
            
            print(f"📃 FASTA Header: {fasta_lines[0]}")
        
        return mrna_sequences  # Return dictionary {Accession ID: mRNA sequence}
    
    except Exception as e:
        print(f"🚨 Error: {e}")
        return {}


species = "Saccharomyces cerevisiae"
gene = "ACT1"
mrna_seqs = get_mrna_sequence(species, gene)
for acc, seq in mrna_seqs.items():
    print(f"{acc}: {seq}")
    print(len(seq))  # print the first 60 bases for brevity


🔍 Search Query: Saccharomyces cerevisiae[Organism] AND ACT1[Gene] AND biomol_mrna[PROP] AND srcdb_refseq[PROP]
✅ Found IDs: ['296144393']
📥 Fetching sequence for ID: 296144393...
📃 FASTA Header: >NM_001179927.1 Saccharomyces cerevisiae S288C actin (ACT1), partial mRNA
NM_001179927.1: AUGGAUUCUGAGGUUGCUGCUUUGGUUAUUGAUAACGGUUCUGGUAUGUGUAAAGCCGGUUUUGCCGGUGACGACGCUCCUCGUGCUGUCUUCCCAUCUAUCGUCGGUAGACCAAGACACCAAGGUAUCAUGGUCGGUAUGGGUCAAAAAGACUCCUACGUUGGUGAUGAAGCUCAAUCCAAGAGAGGUAUCUUGACUUUACGUUACCCAAUUGAACACGGUAUUGUCACCAACUGGGACGAUAUGGAAAAGAUCUGGCAUCAUACCUUCUACAACGAAUUGAGAGUUGCCCCAGAAGAACACCCUGUUCUUUUGACUGAAGCUCCAAUGAACCCUAAAUCAAACAGAGAAAAGAUGACUCAAAUUAUGUUUGAAACUUUCAACGUUCCAGCCUUCUACGUUUCCAUCCAAGCCGUUUUGUCCUUGUACUCUUCCGGUAGAACUACUGGUAUUGUUUUGGAUUCCGGUGAUGGUGUUACUCACGUCGUUCCAAUUUACGCUGGUUUCUCUCUACCUCACGCCAUUUUGAGAAUCGAUUUGGCCGGUAGAGAUUUGACUGACUACUUGAUGAAGAUCUUGAGUGAACGUGGUUACUCUUUCUCCACCACUGCUGAAAGAGAAAUUGUCCGUGACAUCAAGGAAAAACUAUGUUACGUCGCCUUGGACUUCGAACAAGAAAUGCAAACCGCUGCUCAAUCUUCUUCAAUUGAAAAAU

In [27]:
from Bio import Entrez
Entrez.email = "qamilmirza@berkeley.edu"

def get_mrna_sequence_with_debug(species_name, gene_name):
    """
    Fetches the mRNA sequence for a given species and gene.
    Prints out the raw responses from the Entrez queries for debugging.
    
    Parameters:
      species_name (str): e.g., "Homo sapiens" or "Saccharomyces cerevisiae"
      gene_name (str): e.g., "INS" or "CYC1"
      
    Returns:
      A single mRNA sequence (string) with T's replaced by U's,
      or an empty string if no sequence is found.
    """
    try:
        # --- First Query: Using the refined query with refseq_mrna filter ---
        search_query = f"{species_name}[Organism] AND {gene_name}[Gene] AND refseq_mrna[Filter]"
        print("Using refined query:")
        print(search_query)
        
        handle = Entrez.esearch(db="nucleotide", term=search_query, retmax=5)
        record = Entrez.read(handle)
        handle.close()
        
        print("\nRaw esearch response (refseq_mrna filter):")
        print(record)
        
        # If no records are found, fall back to a generic query using mRNA filter
        if not record["IdList"]:
            search_query = f"{species_name}[Organism] AND {gene_name}[Gene] AND mRNA[Filter]"
            print("\nNo records found with refseq_mrna filter.")
            print("Using fallback query:")
            print(search_query)
            
            handle = Entrez.esearch(db="nucleotide", term=search_query, retmax=5)
            record = Entrez.read(handle)
            handle.close()
            
            print("\nRaw esearch response (fallback mRNA filter):")
            print(record)
            
            if not record["IdList"]:
                print("\nNo mRNA records found with either query.")
                return ""
        
        # Select the first gene ID (assumed to be the canonical transcript)
        gene_id = record["IdList"][0]
        print("\nSelected Gene ID:")
        print(gene_id)
        
        # --- Fetch the FASTA sequence for the selected record ---
        fetch_handle = Entrez.efetch(db="nucleotide", id=gene_id, rettype="fasta", retmode="text")
        fasta_data = fetch_handle.read()
        fetch_handle.close()
        
        print("\nRaw FASTA data retrieved:")
        print(fasta_data)
        
        # Process FASTA data: remove header and join sequence lines
        fasta_lines = fasta_data.strip().split("\n")
        sequence_only = ''.join(fasta_lines[1:])
        mrna_sequence = sequence_only.replace(" ", "").replace("T", "U").replace("t", "u")
        
        return mrna_sequence
    except Exception as e:
        print("Error:", e)
        return ""

# Example usage in your Jupyter Notebook:
species_name = "Saccharomyces cerevisiae"
gene_name = "CYC1"

print("Fetching mRNA sequence for:")
print(f"Species: {species_name}, Gene: {gene_name}\n")

mrna_sequence = get_mrna_sequence_with_debug(species_name, gene_name)

if mrna_sequence:
    print("\nFinal Processed mRNA Sequence (first 100 nucleotides):")
    print(mrna_sequence[:100])
    print("\nTotal Sequence Length:", len(mrna_sequence), "nucleotides")
else:
    print("No sequence retrieved.")


Fetching mRNA sequence for:
Species: Saccharomyces cerevisiae, Gene: CYC1

Using refined query:
Saccharomyces cerevisiae[Organism] AND CYC1[Gene] AND refseq_mrna[Filter]

Raw esearch response (refseq_mrna filter):
{'Count': '0', 'RetMax': '0', 'RetStart': '0', 'IdList': [], 'TranslationSet': [{'From': 'Saccharomyces cerevisiae[Organism]', 'To': '"Saccharomyces cerevisiae"[Organism]'}], 'QueryTranslation': 'Saccharomyces cerevisiae[Organism] AND CYC1[Gene] AND refseq_mrna[Filter]', 'ErrorList': {'PhraseNotFound': ['refseq_mrna[Filter]'], 'FieldNotFound': []}, 'WarningList': {'OutputMessage': ['No items found.'], 'QuotedPhraseNotFound': [], 'PhraseIgnored': []}}

No records found with refseq_mrna filter.
Using fallback query:
Saccharomyces cerevisiae[Organism] AND CYC1[Gene] AND mRNA[Filter]
Error: Search Backend failed: 		

No sequence retrieved.


In [16]:
from Bio import Entrez

Entrez.email = "qamilmirza@berkeley.edu"

def get_gene_names(species_name):
    """
    Fetch gene names for a species from NCBI.

    Args:
        species_name (str): e.g., "Homo sapiens"

    Returns:
        list: Sorted list of gene names.
    """
    try:
        # Search the ncbi gene database for gene names pertaining to the species but only take the first 30 for brevity
        search_query = f"{species_name}[Organism]"
        handle = Entrez.esearch(db="gene", term=search_query, retmax=30)
        record = Entrez.read(handle)
        handle.close()

        print(record)

        # If we cannot find anything, then just return empty list
        if not record["IdList"]:
            return []

        # otgerwise get the gene ids
        gene_ids = record["IdList"]

        # Then collect all the gene names using the gene ids to query the database
        gene_names = []
        for gene_id in gene_ids:
            summary_handle = Entrez.esummary(db="gene", id=gene_id)
            summary_record = Entrez.read(summary_handle)
            print(summary_record)
            summary_handle.close()
            if "DocumentSummarySet" in summary_record:
                doc_summary = summary_record["DocumentSummarySet"]["DocumentSummary"]
                if doc_summary:
                    gene_names.append(doc_summary[0]["Name"])

        # I used a set to prevent duplicates and then sorted the list
        return sorted(set(gene_names))
    except Exception as e:
        print("Error:", e)
        return []

In [20]:
names = get_gene_names("Homo sapiens")

{'Count': '360066', 'RetMax': '30', 'RetStart': '0', 'IdList': ['139440227', '139440225', '139440223', '139440221', '139440219', '139440217', '139440215', '139432819', '139432817', '139432815', '139432813', '139432811', '139432809', '139432807', '139432805', '139432803', '139432801', '139427322', '139355254', '139355252', '139355250', '139355248', '139355243', '139355241', '139355239', '139355237', '139355235', '139355233', '139355231', '139355229'], 'TranslationSet': [{'From': 'Homo sapiens[Organism]', 'To': '"Homo sapiens"[Organism]'}], 'TranslationStack': [{'Term': '"Homo sapiens"[Organism]', 'Field': 'Organism', 'Count': '360066', 'Explode': 'Y'}, 'GROUP'], 'QueryTranslation': '"Homo sapiens"[Organism]'}
{'DocumentSummarySet': DictElement({'DocumentSummary': [DictElement({'Name': 'NACC1-AS1', 'Description': 'NACC1 antisense RNA 1', 'Status': '0', 'CurrentID': '0', 'Chromosome': '19', 'GeneticSource': 'genomic', 'MapLocation': '19p13.13', 'OtherAliases': '', 'OtherDesignations': '',

KeyboardInterrupt: 

In [11]:
names

[]